This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset.
   * No visibility sample noise. Just testing that things are working exactly as expected.
   * Using the GLEAM sky model and a common everybeam station beam model.
   * Add complex Gaussian noise corruptions to station Jones matrices.
   * Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities (using GLEAM and everybeam).$^{*}$
* Do bandpass calibration.$^{*}$
* Check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with only gain corruptions and a gain-only solver,
then again with gain and leakage corruptions and a polarised solver.

In [1]:
from time import perf_counter

from ska_sdp_instrumental_calibration.processing_tasks.lsm import Component
from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
gleamfile = "/data/EOS_1/mit183/gleamegc.dat"
eb_ms = "/data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms"
eb_coeffs = "/data/EOS_1/mit183/SKA/SP-4626/ska-sdp-func-everybeam/coeffs"

In [3]:
# Simple bandpass calibration of X and Y gains with a user-defined sky model
t0 = perf_counter()
bandpass_calibration.run(
    {
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, Fint200=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, Fint200=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-17 18:49:41,838 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-17 18:49:41,839 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2024-12-17 18:49:41,840 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-17 18:49:41,867 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-17 18:49:42,453 - processing_tasks.predict - INFO - No beam model used in predict
2024-12-17 18:49:42,612 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-17 18:49:43,120 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-17 18:49:43,121 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-17 18:49:43,946 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-17 18:49:44,085 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12

pipeline ran in 4.4 seconds


In [4]:
# Simple bandpass calibration of X and Y gains with GLEAM and EveryBeam
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-17 18:49:46,196 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-17 18:49:46,198 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2024-12-17 18:49:46,199 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-17 18:49:46,200 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2024-12-17 18:49:46,201 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-17 18:49:46,202 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2024-12-17 18:49:46,203 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2024-12-17 18:49:46,840 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2024-12-17 18:49:46,841 - workflow.pipeline_config - INFO - LSM: found 37 components
2024-12-17 18:49:46,851 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-17 18:49:46,882 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-17 18:49:50,162 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-17 18:49:50,751 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-17 18:49:50,752 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-17 18:49:51,576 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-17 18:49:51,644 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2024-12-17 18:49:51,646 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-17 18:49:51,647 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2024-12-17 18:49:51,648 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2024-12-17 18:49:52,339 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2024-12-17 18:49:52,340 - pipeline.bandpass_calibration - INFO - LSM: found 37 components
2024-12-17 

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-17 18:49:56,177 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
2024-12-17 18:49:56,197 - pipeline.bandpass_calibration - INFO - Checking results
2024-12-17 18:49:56,200 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 10.4 seconds


In [5]:
# Bandpass calibration with leakage
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
        "gains": True,
        "leakage": True,
        "solver": "normal_equations",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-17 18:49:56,605 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-17 18:49:56,606 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2024-12-17 18:49:56,607 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-17 18:49:56,609 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2024-12-17 18:49:56,609 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-17 18:49:56,609 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2024-12-17 18:49:56,610 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2024-12-17 18:49:57,231 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2024-12-17 18:49:57,233 - workflow.pipeline_config - INFO - LSM: found 37 components
2024-12-17 18:49:57,258 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-17 18:49:57,304 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-17 18:50:00,298 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-17 18:50:00,300 - workflow.utils - INFO - Applying direction-independent leakage corruptions
2024-12-17 18:50:00,868 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-17 18:50:00,869 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-17 18:50:01,635 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-17 18:50:01,711 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2024-12-17 18:50:01,713 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-17 18:50:01,714 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2024-12-17 18:50:01,715 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2024-12-17 18:50:02,478 - processing_tasks.lsm - INFO - extracted 37 GLEAM components


Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-17 18:50:07,795 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
2024-12-17 18:50:07,809 - pipeline.bandpass_calibration - INFO - Checking results
2024-12-17 18:50:07,811 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 11.6 seconds
